# Loading data

Let's with NECOFS data, available at [SMAST archive](http://www.smast.umassd.edu:8080/thredds/catalog/models/fvcom/NECOFS/Archive/NECOFS_GOM/catalog.html).

In [1]:
#importing modules
import xarray as xr
from netCDF4 import Dataset

We need to drop `siglay` and `siglev` coordinates. For some reason the data have the same name for coordinates and dimensions, e.g `siglay` is a coordinate that depends on `(siglay,node)`. [Many others](https://github.com/pydata/xarray/issues/2233) had the same problem with this model output. There is also some problem with the time formatting, we will need to decode it after loading the data.

In [42]:
# define year and month to be read
year = 2019
month = 5
# we could use this to run a loop through the years/months we need


# list problematic coordinates
drop_variables = ['siglay','siglev']

# base url for openDAP server
url = "".join(["http://www.smast.umassd.edu:8080/thredds/dodsC/models/fvcom/",
        f"NECOFS/Archive/NECOFS_GOM/{year}/gom4_{year}{month:02d}.nc?"])

# lazy load of the data
ds = xr.open_dataset(url,drop_variables=drop_variables,decode_times=False)

Solving the problem with `siglay` and `siglev`. We need to workaround using `netCDF4` and renaming the coordinates.

In [43]:
# load data with netCDF4
nc = Dataset(url)
# load the problematic coordinates
coords = {name:nc[name] for name in drop_variables}

# function to extract ncattrs from `Dataset()`
get_attrs = lambda name: {attr:coords[name].getncattr(attr) for attr in coords[name].ncattrs()}
# function to convert from `Dataset()` to `xr.DataArray()`
nc2xr = lambda name: xr.DataArray(coords[name],attrs=get_attrs(name),name=f'{name}_coord',dims=(f'{name}','node'))

# apply `nc2xr()` and merge `xr.DataArray()` objects
coords = xr.merge([nc2xr(name) for name in coords.keys()])

# reassign to the main `xr.Dataset()`
ds = ds.assign_coords(coords)

We now can assign the `z` coordinate for the data.

In [68]:
z = (ds.siglay_coord*ds.h)
z.attrs = dict(long_name='nodal z-coordinate',units='meters')
ds = ds.assign_coords(z=z)